# Homework Week 2

Simulation and Resampling

- MARCOS ÁLVAREZ MARTÍN
- SIMON SCHMETZ

# Excercise 1

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# get first 50 rows
iris_setosa_df = iris_df.iloc[:50]

iris_setosa_df.head()

## Shapiro-Wilk Normality Test

First, we perform the Shapiro-Wilk normality test to verify the normality of the data with

$H_0:$ Data is normaly distributed

$H_1:$ Data is not normaly distributed

With the following resulting P-Values. We se that with significane 5%, for all features except for the petal width, the null hypothesis is not rejected. The petal length P-Value however is however not as high as it is for sepal width and length. 

In [ ]:
import scipy.stats as stats

# perform iris_setosa_df test

print("P-Values of Shapiro Wilk normality test")
for column in iris_setosa_df.columns:
    stat, p_value = stats.shapiro(iris_setosa_df[column])
    print(f"{column}: {p_value}")


To Validate the findings, histograms together with fitted normal distributions show how well the data fits or does not fit a normal distribution. The Strong rejection of normality for petal width is confirmed by the corresponding histogram. 

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# set up fig
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
axs = axs.flatten()

# plot each column
for i, column in enumerate(iris_setosa_df.columns):
    data = iris_setosa_df[column]
    
    # plot hist
    axs[i].hist(data, bins=10, density=True, alpha=0.6, color='g')
    
    # Plot normal distribution fitted to the data
    mu, std = norm.fit(data)
    xmin, xmax = axs[i].get_xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    axs[i].plot(x, p, 'k', linewidth=2)
    
    # cosmetics
    axs[i].set_title(f'Histogram and Normal Distribution of {column}')
    axs[i].set_xlabel(column)
    axs[i].set_ylabel('Density')

# plot
plt.tight_layout()
plt.show()

## Kolmogorov-Smirnov Test

Alternatively, the Kolmogorov-Smirnov Test can be used to perform a similar evaluation of nomrality. As Kolmogorov-Smirnov is a nonparametric test, we need to give a reference distribution which in our case is the normal distribution with parameters mean and variance taken as the sample mean and the sample variance. 

The test gives a similar result as the Shapiro-Wilk test, with a general slight increase in P-Values for all Features. The conclusions drawn previously however remain the same. 

In [ ]:
# Perform Kolmogorov-Smirnov test
print("P-Values of Kolmogorov-Smirnov test with normal distribution as reference")

teststat_values_iris = {}


for column in iris_setosa_df.columns:
    # fit normal distribution & perform test
    mu, std = norm.fit(iris_setosa_df[column])
    ks_statistic, ks_p_value = stats.kstest(iris_setosa_df[column], 'norm', args=(mu, std))

    # store test staitic value
    teststat_values_iris[column] = ks_statistic

    # print
    print(f"{column}: {ks_p_value}")

## Simulation

For a sample size of 50, we perform 1000 Monte Carlo Simulations by drawing random samples from a normal distribution, estimating the parameters of a normal distribution on each of those samples and performing a Kolmogorov-Smirnov with a normal distribution defined by the estimated parameters als reference. The test results (test statistic value and p-Value) are stored. For further analysis, the P-Values are plotted as histograms, showing how the majority of Tests have P-Values close to 1.

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

# Vars
M = 1000  # Number of simulations
n = 50  # Number of observations

ks_statistics = []
p_values = []


# Simulation
for _ in range(M):
    # Generate sample and estimate params
    sample = np.random.normal(loc=0, scale=1, size=n)
    mu, std = norm.fit(sample)
    
    # Perform test
    ks_statistic, p_value = stats.kstest(sample, 'norm', args=(mu, std))
    
    # store test values
    ks_statistics.append(ks_statistic)
    p_values.append(p_value)

# plot p-Values
plt.hist(p_values)

## Approximates the p-value of the normality Kolmogorov-Smirnov test

By comparing the P-Values from the real dataset with the P-Values from simulated samples, we can approximate the P-Value of the normality Kolmogorov-Smirnov test, e.g. the probability that a even greater test statistic could come from a random sample of a normal distribution. The resulting "P-Values" for sepal length and width as well as for petal width are in line with previous evaluation of normality. Meanwhile, petal length has contrary to previous findings a lower P-Value impying non-normality. 

In [ ]:

percentages = {}
for column, test_stat in teststat_values_iris.items():
    count = sum(1 for stat in ks_statistics if stat > test_stat)
    percentage = (count / len(ks_statistics))
    percentages[column] = percentage

percentages

The findings can be validated by plotting the distirbution of the test statistic for the simulation and the test statistic values calculated for the features.

In [ ]:
# make histogram
plt.hist(ks_statistics, bins=30, alpha=0.7, color='blue', edgecolor='black')
colors = ['red', 'green', 'orange', 'purple']

for i, column in enumerate(iris_setosa_df.columns):
    plt.axvline(teststat_values_iris[column], color=colors[i], linestyle='dashed', linewidth=2, label=f'Test statistic {column}')

plt.legend()
plt.xlabel('KS Statistic')
plt.ylabel('Frequency')
plt.show()

# Excercise 2

# Excercise 3 - Building Confidence Interval

In the following Excercise, calculate the required number of observations to build a 99% CI on $\int_{0}^{1} \exp(e^x) \, dx$ with a width of 0.05 Units. We do so by first simulating 1000 observations drawn from a Uniform and estimating the standard deviation of $\exp(e^x)$. 


In [58]:

M = 1000

# draw sample and calculate std
U = np.random.uniform(0, 1, M)
exp_U = np.exp(U)
std_exp_U = np.std(exp_U, ddof=1)

Then we use the the dependence between CI length ($l_{CI}$) and number of observations ($n$) 
$$2*\pm \text{z} \cdot \frac{\sigma_U}{\sqrt{n}}=l_{CI}$$
$$ n = \left( \frac{2 \cdot \text{z} \cdot \sigma_U}{l_{CI}} \right)^2$$
to calculate the required number of observations.

In [62]:
# calculate params
z_score = stats.norm.ppf(0.995) 
desired_width = 0.05

# calculate required n
required_n = (2 * z_score * std_exp_U / desired_width)^2

print(f"Number of observations needed: {int(np.ceil(required_n))}")

# Excercise 4